# Tạo **Index gộp** cho nhiều bộ dữ liệu

- **Mục đích**
  - Hợp nhất **tất cả entries** từ nhiều thư mục nguồn (mỗi thư mục là *dataset + split*) thành **một file index duy nhất** cho từng bộ.
  - Mỗi entry ánh xạ `sentence_text` ↔ `vector_file_path` (vector đã **chuẩn hoá**) để phục vụ truy vấn/huấn luyện/đánh giá.


- **Đầu vào**
  - Danh sách cấu hình `datasets_to_process` (mỗi phần tử là bộ 3 tham số):  
    `(<name>, <data_dir>, <vectors_dir>)`
    ```python
    [
      ("finetuned_gramvar_train",
        ".../Clean_Dataset/Corpus/Split_GramVar/Train",
        ".../finetuned_train_vectors_normalized_gramvar"),
      ("finetuned_parave_train",
        ".../Clean_Dataset/Corpus/Split_ParaVE/Train",
        ".../finetuned_train_vectors_normalized_parave"),
      # có thể bổ sung thêm pre-trained / val / test ...
    ]
    ```
  - Thư mục **dữ liệu gốc**: nhiều file JSON theo từng động từ, dạng  
    `*_train_set.json` → mỗi file là **list** các item có khoá `"text"`.
  - Thư mục **vector đã chuẩn hoá** theo động từ:  
    `<vectors_dir>/<verb_name>/sentence_<i>.pt`

- **Đầu ra**
  - Thư mục đích:  
    `.../consolidated_indexes/`
  - Với mỗi cấu hình `<name>` sinh **01 file**:
    ```
    <name>_consolidated_index.json
    ```
  - Nội dung file là **một danh sách (list)** các entry:
    ```json
    [
      {
        "sentence_text": "Protein A interacts with protein B.",
        "verb_name": "interact",
        "vector_file_path": "/content/.../finetuned_train_vectors_normalized_gramvar/interact/sentence_0.pt"
      },
      {
        "sentence_text": "Complex C binds to DNA.",
        "verb_name": "bind",
        "vector_file_path": "/content/.../finetuned_train_vectors_normalized_gramvar/bind/sentence_1.pt"
      }
    ]
    ```

- **Quy trình**
  1. Với từng bộ trong `datasets_to_process`:
     - **Bỏ qua** nếu file gộp đã tồn tại.
     - **Quét** tất cả `*.json` trong `<data_dir>`.
     - **Suy ra** `verb_name` từ tên file: `*_train_set.json` → `<verb_name>`.
     - **Đọc** list câu từ khoá `"text"` (lọc rỗng).
     - **Ghép** đường dẫn vector tương ứng:  
       `<vectors_dir>/<verb_name>/sentence_<i>.pt`
     - **Ghi** toàn bộ entries vào `.../consolidated_indexes/<name>_consolidated_index.json`.

In [ ]:
# Import các thư viện cần thiết
import os
import json
import glob

# THIẾT LẬP CÁC ĐƯỜNG DẪN

# Đường dẫn thư mục gốc trên Google Drive
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# DANH SÁCH CÁC BỘ DỮ LIỆU CẦN TẠO INDEX GỘP
# Mỗi mục là một tuple chứa: (tên định danh, đường dẫn dữ liệu gốc, đường dẫn vector tương ứng)
datasets_to_process = [
    (
        "finetuned_gramvar_train",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train'),
        os.path.join(drive_base_path, 'finetuned_train_vectors_normalized_gramvar')
    ),
    (
        "finetuned_parave_train",
        os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train'),
        os.path.join(drive_base_path, 'finetuned_train_vectors_normalized_parave')
    ),
]

# Đường dẫn đến thư mục MỚI để lưu các file index đã gộp
output_dir = os.path.join(drive_base_path, 'consolidated_indexes')
os.makedirs(output_dir, exist_ok=True)

# THỰC THI CHƯƠNG TRÌNH TẠO INDEX GỘP

if __name__ == "__main__":
    print("--- Bắt đầu quá trình tạo index gộp từ dữ liệu gốc ---")

    for name, data_dir, vectors_dir in datasets_to_process:
        output_file_path = os.path.join(output_dir, f"{name}_consolidated_index.json")

        if os.path.exists(output_file_path):
            print(f"File gộp '{os.path.basename(output_file_path)}' đã tồn tại. Bỏ qua.")
            continue

        print(f"\nĐang xử lý bộ dữ liệu: {name}")

        all_index_entries = []
        # Tìm tất cả các file .json trong thư mục dữ liệu gốc (ví dụ: .../Train)
        json_files = glob.glob(os.path.join(data_dir, '*.json'))

        for json_file in json_files:
            # Lấy tên động từ
            base_name = os.path.basename(json_file)
            verb_name = "_".join(base_name.replace('_train_set.json', '').split('_'))

            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Trích xuất các câu
            sentences = [item['text'] for item in data if 'text' in item and item['text'].strip()]

            # Tạo các mục index
            for i, sentence_text in enumerate(sentences):
                vector_file_path = os.path.join(vectors_dir, verb_name, f"sentence_{i}.pt")

                index_entry = {
                    "sentence_text": sentence_text,
                    "verb_name": verb_name,
                    "vector_file_path": vector_file_path
                }
                all_index_entries.append(index_entry)

        # Lưu danh sách đã gộp vào một file duy nhất
        if all_index_entries:
            with open(output_file_path, 'w', encoding='utf-8') as f:
                json.dump(all_index_entries, f, indent=4, ensure_ascii=False)
            print(f" -> Đã tạo index gộp với {len(all_index_entries)} mục và lưu thành công.")
        else:
            print(f" -> Không có dữ liệu để xử lý trong '{name}'.")

    print("\n--- QUÁ TRÌNH TẠO INDEX GỘP ĐÃ HOÀN TẤT! ---")

--- Bắt đầu quá trình tạo index gộp từ dữ liệu gốc ---
File gộp 'finetuned_gramvar_train_consolidated_index.json' đã tồn tại. Bỏ qua.
File gộp 'finetuned_parave_train_consolidated_index.json' đã tồn tại. Bỏ qua.

--- QUÁ TRÌNH TẠO INDEX GỘP ĐÃ HOÀN TẤT! ---


- **Cấu trúc thư mục (minh hoạ)**
  ```bash
  .../Clean_Dataset/Corpus/Split_GramVar/Train/
  ├── begin_1_train_set.json
  ├── bind_2_train_set.json
  └── ...

  .../finetuned_train_vectors_normalized_gramvar/
  ├── begin_1/
  │   ├── sentence_0.pt
  │   ├── sentence_1.pt
  │   └── ...
  ├── bind_2/
  │   ├── sentence_0.pt
  │   └── ...
  └── ...

  .../consolidated_indexes/
  ├── finetuned_gramvar_train_consolidated_index.json
  └── finetuned_parave_train_consolidated_index.json